### Student Information
Name: 李庭競

Student ID: 111033626

GitHub ID: CrazyRyan0812

Kaggle name: CrazyRyan

Kaggle private scoreboard snapshot:

![Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__. 

----
# 1 Data Preperation
## 1.1 read data to panda_dataframe

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import json

data_identification = pd.read_csv("dm2022-isa5810-lab2-homework/data_identification.csv", skiprows=1, header=None, names=["tweet_id", "identification"])
emotion = pd.read_csv("dm2022-isa5810-lab2-homework/emotion.csv", skiprows=1, header=None, names=["tweet_id", "emotion"])

tweets_DM_json=[]
for line in open('dm2022-isa5810-lab2-homework/tweets_DM.json', 'r'):
    tweets_DM_json.append(json.loads(line))

In [3]:
extract_tweets = [(tw['_source']['tweet']['tweet_id'],tw['_source']['tweet']['text'],tw['_source']['tweet']['hashtags'],tw['_score']) for tw in tweets_DM_json]

In [4]:
tweets_PD = pd.DataFrame(extract_tweets, columns=['tweet_id', 'text', 'hashtag','_score'])
tweets_PD

,tweet_id,text,hashtag,_score
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],232
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[],989
...,...,...,...,...
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,"[mixedfeeling, butimTHATperson]",827
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,[],368
1867532,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,[],498
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840


In [5]:
train_data_identification_df = data_identification.loc[data_identification["identification"] == 'train']
train_data_identification_list = list(train_data_identification_df['tweet_id'])
test_data_identification_df = data_identification.loc[data_identification["identification"] == 'test']
test_data_identification_list = list(test_data_identification_df['tweet_id'])

In [6]:
train_dataset = pd.merge(tweets_PD, emotion, on='tweet_id')
ORI_train_dataset = pd.merge(tweets_PD, emotion, on='tweet_id')
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


In [7]:
test_dataset = pd.merge(test_data_identification_df, tweets_PD, on='tweet_id')
test_dataset = test_dataset.drop(['identification'], axis=1)
test_dataset

,tweet_id,text,hashtag,_score
0,0x28cc61,@Habbo I've seen two separate colours of the e...,[],107
1,0x2db41f,@FoxNews @KellyannePolls No serious self respe...,[],728
2,0x2466f6,"Looking for a new car, and it says 1 lady owne...",[womendrivers],491
3,0x23f9e9,@cineworld “only the brave” just out and fount...,[robbingmembers],28
4,0x1fb4e1,Felt like total dog 💩 going into open gym and ...,[],925
...,...,...,...,...
411967,0x2c4dc2,6 year old walks in astounded. Mum! Look how b...,[kids],792
411968,0x31be7c,Only one week to go until the #inspiringvolunt...,[inspiringvolunteerawards2017],34
411969,0x1ca58e,"I just got caught up with the manga for ""My He...",[],976
411970,0x35c8ba,Speak only when spoken to and make hot ass mus...,[],534


## 1.2 data cleaning

### 1.2.1 Mis-spelling data

Once I try to search some stategy to clean data, I found some useful dataset on kaggle.


https://www.kaggle.com/datasets/bittlingmayer/spelling

In [8]:
misspell_data = pd.read_csv("data_cleaning_dataset/aspell.txt", sep = ":", names = ["correction","misspell"])
misspell_data
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

In [9]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

train_dataset["text"] = train_dataset["text"].apply(lambda x : misspelled_correction(x))
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.2 Convert emoji to text
I find a tutorial on this website, and use the dictionary from it.

https://medium.com/geekculture/a-tutorial-on-performing-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk-40e5b35ab440

use pip install emoji to complete it

In [10]:
def load_dict_smileys():
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

In [11]:
# around 30s.
import emoji
def remove_emoticons(tweet):
    smilies = load_dict_smileys()
    split_tweet = tweet.split(" ")
    for key,val in smilies.items():
        if key in tweet:
            new_tweet = tweet.replace(key, val)
            tweet = new_tweet
            tweet = emoji.demojize(tweet)
    return tweet

train_dataset["text"] = train_dataset["text"].apply(lambda x: remove_emoticons(x))
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.3 Remove URLS
use pip install mysmallutils to complete it

In [12]:
# around 10s.
from mysutils.text import remove_urls
URL_PATTERN = r'[A-Za-z0-9]+://[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'
train_dataset["text"] = train_dataset["text"].apply(lambda x: remove_urls(x))
URL_PATTERN = r'[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'
train_dataset["text"] = train_dataset["text"].apply(lambda x: remove_urls(x))
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.4 Remove twitter mention
use pandas.dataframe.str.replace to complete it

In [13]:
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*@\w+', '', regex=True)
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*\B@\w+', '', regex=True)
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*@\S+', '', regex=True)
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*@\S+\b', '', regex=True)
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"As we see, Trump is dangerous to #freepress a...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,Thx for the BEST TIME tonight. What stories! ...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.5 Remove twitter <<LH>>
use pandas.dataframe.str.replace to complete it

In [14]:
train_dataset['text'] = train_dataset['text'].str.replace(r'<LH>', '', regex=True)
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"As we see, Trump is dangerous to #freepress a...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂,[],376,fear
3,0x1d755c,Thx for the BEST TIME tonight. What stories! ...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus.,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date using ...",[],840,joy


### 1.2.6 Remove Stopwords

Here, I use the nltk stopwords to implement it

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

train_dataset['text'] = train_dataset['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to /home/tclee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(train_dataset, test_size=0.2)
ORI_df_train, ORI_df_valid = train_test_split(ORI_train_dataset, test_size=0.2)

# 2 Data tokenize
I use 3 ways to tokenize the data

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)
BOW_500.fit(df_train['text'])

X_train_BOW500 = BOW_500.transform(df_train['text'])
X_valid_BOW500 = BOW_500.transform(df_valid['text'])
y_train_BOW500 = df_train['emotion']
y_valid_BOW500 = df_valid['emotion']

test_bow500 = BOW_500.transform(test_dataset['text'])

/home/tclee/anaconda3/envs/Data_mining/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [18]:
BOW_2000 = CountVectorizer(max_features=2000, tokenizer=nltk.word_tokenize)
BOW_2000.fit(df_train['text'])

X_train_BOW2000 = BOW_2000.transform(df_train['text'])
X_valid_BOW2000 = BOW_2000.transform(df_valid['text'])
y_train_BOW2000 = df_train['emotion']
y_valid_BOW2000 = df_valid['emotion']

test_bow2000 = BOW_2000.transform(test_dataset['text'])

In [19]:
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)
BOW_500.fit(ORI_df_train['text'])

X_train_BOW500_ORI = BOW_500.transform(ORI_df_train['text'])
X_valid_BOW500_ORI = BOW_500.transform(ORI_df_valid['text'])
y_train_BOW500_ORI = ORI_df_train['emotion']
y_valid_BOW500_ORI = ORI_df_valid['emotion']

test_bow500_ORI = BOW_500.transform(test_dataset['text'])

In [20]:
BOW_2000 = CountVectorizer(max_features=2000, tokenizer=nltk.word_tokenize)
BOW_2000.fit(ORI_df_train['text'])

X_train_BOW2000_ORI = BOW_2000.transform(ORI_df_train['text'])
X_valid_BOW2000_ORI = BOW_2000.transform(ORI_df_valid['text'])
y_train_BOW2000_ORI = ORI_df_train['emotion']
y_valid_BOW2000_ORI = ORI_df_valid['emotion']

test_bow2000_ORI = BOW_2000.transform(test_dataset['text'])

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.corpus import stopwords
# twt_tknzr = nltk.tokenize.TweetTokenizer(strip_handles=True, reduce_len=True)
# tfidf_vector = TfidfVectorizer(max_features=500, tokenizer=twt_tknzr.tokenize)
tfidf_vector = TfidfVectorizer(max_features=500) 

tfidf_vector.fit(ORI_df_train['text'])
X_train_tfidf_ORI = tfidf_vector.fit_transform(ORI_df_train['text'])
X_valid_tfidf_ORI = tfidf_vector.fit_transform(ORI_df_valid['text'])
y_train_tfidf_ORI = ORI_df_train['emotion']
y_valid_tfidf_ORI = ORI_df_valid['emotion']

test_tfidf_tfidf_ORI = tfidf_vector.transform(test_dataset['text'])

In [22]:
tfidf_vector = TfidfVectorizer(max_features=500) 

tfidf_vector.fit(ORI_df_train['text'])
X_train_tfidf = tfidf_vector.fit_transform(df_train['text'])
X_valid_tfidf = tfidf_vector.fit_transform(df_valid['text'])
y_train_tfidf = df_train['emotion']
y_valid_tfidf = df_valid['emotion']

test_tfidf_tfidf = tfidf_vector.transform(test_dataset['text'])

In [23]:
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(df_train['text'].tolist())
tokenizer.fit_on_texts(df_valid['text'].tolist())
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=100, padding='post')
    return padded_sequences

X_train_keras = get_sequences(tokenizer, df_train['text'].tolist())
X_valid_keras = get_sequences(tokenizer, df_valid['text'].tolist())

emotions = set(df_train['emotion'].tolist())
emotions_to_index = dict((c, i) for i, c in enumerate(emotions))
index_to_emotions = dict((v, k) for k, v in emotions_to_index.items())
names_to_ids = lambda labels: np.array([emotions_to_index.get(x) for x in labels])

y_train_keras = names_to_ids(df_train['emotion'].tolist())
y_valid_keras = names_to_ids(df_valid['emotion'].tolist())

In [24]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(ORI_df_train['text'].tolist())
tokenizer.fit_on_texts(ORI_df_valid['text'].tolist())
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=100, padding='post')
    return padded_sequences

X_train_keras_ORI = get_sequences(tokenizer, ORI_df_train['text'].tolist())
X_valid_keras_ORI = get_sequences(tokenizer, ORI_df_valid['text'].tolist())

emotions = set(df_train['emotion'].tolist())
emotions_to_index = dict((c, i) for i, c in enumerate(emotions))
index_to_emotions = dict((v, k) for k, v in emotions_to_index.items())
names_to_ids = lambda labels: np.array([emotions_to_index.get(x) for x in labels])

y_train_keras_ORI = names_to_ids(ORI_df_train['emotion'].tolist())
y_valid_keras_ORI = names_to_ids(ORI_df_valid['emotion'].tolist())

In [51]:
X_test_keras = get_sequences(tokenizer, test_dataset['text'].tolist())

# 3 Train different models to predict
## 3.1 DecisionTree

In [25]:
#about 50mins
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
model = DecisionTreeClassifier(random_state=1012)
model = model.fit(X_train_BOW500, y_train_BOW500)
y_train_pred = model.predict(X_train_BOW500)
y_test_pred = model.predict(X_valid_BOW500)

accuracy = accuracy_score(y_true=y_train_BOW500, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_BOW500, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.9019683112198892
valid accuracy is  0.39225661512883314


In [26]:
test_bow500_result = model.predict(test_bow500)
test_bow500_result

array(['sadness', 'joy', 'disgust', ..., 'anticipation', 'sadness',
       'disgust'], dtype=object)

In [27]:
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_01_DT_BOW500.csv")
pd.DataFrame(test_bow500_result).to_csv("01_DT_BOW500.csv")

## 3.2 Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier

model_RFC = RandomForestClassifier(bootstrap=True, n_estimators=1000, max_depth=4, random_state=1012)
model_RFC.fit(X_train_BOW500, y_train_BOW500)
y_train_pred = model_RFC.predict(X_train_BOW500)
y_test_pred = model_RFC.predict(X_valid_BOW500)

accuracy = accuracy_score(y_true=y_train_BOW500, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_BOW500, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.36654643823264205
valid accuracy is  0.3642503082995263


In [30]:
test_RFC_result = model_RFC.predict(test_bow500)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_02_RF_BOW500.csv")
pd.DataFrame(test_RFC_result).to_csv("02_RF_BOW500.csv")

## 3.3 MultinomialNB

In [31]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train_tfidf, y_train_tfidf)
y_train_pred = MNB.predict(X_train_tfidf)
y_test_pred = MNB.predict(X_valid_tfidf)

accuracy = accuracy_score(y_true=y_train_tfidf, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_tfidf, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.43456567478208596
valid accuracy is  0.3714021702912615


In [32]:
test_MNB_result = MNB.predict(test_tfidf_tfidf)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_03_MNB_TFIDF500.csv")
pd.DataFrame(test_MNB_result).to_csv("03_MNB_TFIDF500.csv")

In [33]:
MNB_tfidf = MultinomialNB()
MNB_tfidf.fit(X_train_tfidf_ORI, y_train_tfidf_ORI)
y_train_pred = MNB_tfidf.predict(X_train_tfidf_ORI)
y_test_pred = MNB_tfidf.predict(X_valid_tfidf_ORI)

accuracy = accuracy_score(y_true=y_train_tfidf_ORI, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_tfidf_ORI, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.4369745373352226
valid accuracy is  0.3837650671732283


In [34]:
test_MNB_result = MNB_tfidf.predict(test_tfidf_tfidf_ORI)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_04_MNB_TFIDF500_ORI.csv")
pd.DataFrame(test_MNB_result).to_csv("04_MNB_TFIDF500_ORI.csv")

## 3.4  XGBClassifier

In [35]:
# about 102mins
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_tfidf_label = le.fit_transform(y_train_tfidf)

model_XGB_tfidf = XGBClassifier(n_estimators=1500, learning_rate= 0.05, max_depth=5, num_class=8)
model_XGB_tfidf.fit(X_train_tfidf, y_train_tfidf_label)
y_train_pred = model_XGB_tfidf.predict(X_train_tfidf)
y_test_pred = model_XGB_tfidf.predict(X_valid_tfidf)

accuracy = accuracy_score(y_true=y_train_tfidf_label, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_tfidf, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.4714981321654
valid accuracy is  0.0


In [ ]:
# tree=1000, train accuracy is 0.455, valid is unavaliable, about 102 mins.
# tree=1500, train accuracy is 0.455, valid is unavaliable, about 130 mins.

In [36]:
# [0 1 2 3 4 5 6 7], got ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise' 'trust']
int_to_emotion = {'0':'anger','1':'anticipation','2':'disgust','3':'fear','4':'joy','5':'sadness','6':'surprise','7':'trust'}
test_XGB_tfidf_result = model_XGB_tfidf.predict(test_tfidf_tfidf)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_05_XGB_tfidf.csv")
pd.DataFrame(test_XGB_tfidf_result).to_csv("05_XGB_tfidf.csv")


## 3.5 MLP
### BOW500 + clean DATA

In [37]:
import keras
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

label_encoder = LabelEncoder()
label_encoder.fit(y_train_BOW500)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train_BOW500[0:4])
print('\ny_train.shape: ', y_train_BOW500.shape)
print('y_test.shape: ', y_valid_BOW500.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train = label_encode(label_encoder, y_train_BOW500)
Y_test = label_encode(label_encoder, y_valid_BOW500)

print('\n\n## After convert')
print('y_train[0:4]:\n', Y_train[0:4])
print('\ny_train.shape: ', Y_train.shape)
print('y_test.shape: ', Y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 1347521    anticipation
628192          disgust
1365751             joy
1262065           trust
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)


In [38]:
# I/O check
input_shape = X_train_BOW500.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [39]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger
import tensorflow.python.keras.backend as K 
sess = K.get_session()

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=400)(X)
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=200)(H1)
H2 = ReLU()(H1_W2)

# 3nd hidden layer
H2_W3 = Dense(units=100)(H2)
H3 = ReLU()(H2_W3)

# output layer
H3_W4 = Dense(units=output_shape)(H3)
H4 = Softmax()(H3_W4)

model_output = H4

# create model
model = Model(inputs=[model_input], outputs=[model_output])
# opt = keras.optimizers.adam_v2(learning_rate=0.001)

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 12
batch_size = 32
# X_train_reorder = X_train.sort_indices
# training!
history = model.fit(X_train_BOW500, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_valid_BOW500, Y_test))

2022-11-24 02:35:37.574797: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-24 02:35:37.576254: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-11-24 02:35:37.577182: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-11-24 02:35:37.577222: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-24 02:35:37.577260: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
20

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
dense (Dense)                (None, 400)               200400    
_________________________________________________________________
re_lu (ReLU)                 (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 100)               0     

2022-11-24 02:35:40.635625: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2022-11-24 02:35:40.659744: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200000000 Hz


Epoch 1/12


2022-11-24 02:35:41.160832: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


1164416/1164450 [============================>.] - ETA: 0s - loss: 1.4684 - acc: 0.4650

/home/tclee/anaconda3/envs/Data_mining/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1164450/1164450 [==============================] - 205s 176us/sample - loss: 1.4684 - acc: 0.4650 - val_loss: 1.4505 - val_acc: 0.4725
Epoch 2/12
1164450/1164450 [==============================] - 204s 175us/sample - loss: 1.4263 - acc: 0.4798 - val_loss: 1.4363 - val_acc: 0.4767
Epoch 3/12
1164450/1164450 [==============================] - 206s 177us/sample - loss: 1.4058 - acc: 0.4875 - val_loss: 1.4365 - val_acc: 0.4787
Epoch 4/12
1164450/1164450 [==============================] - 206s 177us/sample - loss: 1.3890 - acc: 0.4933 - val_loss: 1.4396 - val_acc: 0.4778
Epoch 5/12
1164450/1164450 [==============================] - 204s 175us/sample - loss: 1.3751 - acc: 0.4988 - val_loss: 1.4454 - val_acc: 0.4761
Epoch 6/12
1164450/1164450 [==============================] - 205s 176us/sample - loss: 1.3624 - acc: 0.5036 - val_loss: 1.4517 - val_acc: 0.4743
Epoch 7/12
1164450/1164450 [==============================] - 207s 177us/sample - loss: 1.3513 - acc: 0.5079 - val_loss: 1.4537 - val_a

In [40]:
pred_result = model.predict(test_bow500.toarray(), batch_size=32)
pd.DataFrame(pred_result).to_csv("06_MLP_BOW500_epoch12.csv")
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_06_MLP_BOW500_epoch12.csv")

### BOW500 + ORI DATA

In [41]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

label_encoder = LabelEncoder()
label_encoder.fit(y_train_BOW500_ORI)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train_BOW500_ORI[0:4])
print('\ny_train.shape: ', y_train_BOW500_ORI.shape)
print('y_test.shape: ', y_valid_BOW500_ORI.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train = label_encode(label_encoder, y_train_BOW500_ORI)
Y_test = label_encode(label_encoder, y_valid_BOW500_ORI)

print('\n\n## After convert')
print('y_train[0:4]:\n', Y_train[0:4])
print('\ny_train.shape: ', Y_train.shape)
print('y_test.shape: ', Y_test.shape)

# I/O check
input_shape = X_train_BOW500_ORI.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 1071432    trust
944000       joy
1350944    trust
444367       joy
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)
input_shape:  500
output_shape:  8


In [42]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger
import tensorflow.python.keras.backend as K 
sess = K.get_session()

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=400)(X)
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=200)(H1)
H2 = ReLU()(H1_W2)

# 3nd hidden layer
H2_W3 = Dense(units=100)(H2)
H3 = ReLU()(H2_W3)

# output layer
H3_W4 = Dense(units=output_shape)(H3)
H4 = Softmax()(H3_W4)

model_output = H4

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

csv_logger = CSVLogger('logs/training_log_07.csv')

# training setting
epochs = 12
batch_size = 32
# X_train_reorder = X_train.sort_indices
# training!
history = model.fit(X_train_BOW500_ORI, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_valid_BOW500_ORI, Y_test))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 400)               200400    
_________________________________________________________________
re_lu_3 (ReLU)               (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               80200     
_________________________________________________________________
re_lu_4 (ReLU)               (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               20100     
_________________________________________________________________
re_lu_5 (ReLU)               (None, 100)               0   

In [43]:
pred_result = model.predict(test_bow500_ORI.toarray(), batch_size=128)
pd.DataFrame(pred_result).to_csv("07_MLP_BOW500_epoch12_ORI.csv")
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_07_MLP_BOW500_epoch12_ORI.csv")

### BOW2000 + clean DATA

In [44]:
import keras
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

label_encoder = LabelEncoder()
label_encoder.fit(y_train_BOW2000)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train_BOW2000[0:4])
print('\ny_train.shape: ', y_train_BOW2000.shape)
print('y_test.shape: ', y_valid_BOW2000.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train = label_encode(label_encoder, y_train_BOW2000)
Y_test = label_encode(label_encoder, y_valid_BOW2000)

print('\n\n## After convert')
print('y_train[0:4]:\n', Y_train[0:4])
print('\ny_train.shape: ', Y_train.shape)
print('y_test.shape: ', Y_test.shape)

# I/O check
input_shape = X_train_BOW2000.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 1347521    anticipation
628192          disgust
1365751             joy
1262065           trust
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)
input_shape:  2000
output_shape:  8


In [45]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger
import tensorflow.python.keras.backend as K 
sess = K.get_session()

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=400)(X)
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=200)(H1)
H2 = ReLU()(H1_W2)

# 3nd hidden layer
H2_W3 = Dense(units=100)(H2)
H3 = ReLU()(H2_W3)

# output layer
H3_W4 = Dense(units=output_shape)(H3)
H4 = Softmax()(H3_W4)

model_output = H4

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

csv_logger = CSVLogger('logs/training_log_08.csv')

# training setting
epochs = 4
batch_size = 32
# X_train_reorder = X_train.sort_indices
# training!
history = model.fit(X_train_BOW2000, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_valid_BOW2000, Y_test))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 2000)]            0         
_________________________________________________________________
dense_8 (Dense)              (None, 400)               800400    
_________________________________________________________________
re_lu_6 (ReLU)               (None, 400)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 200)               80200     
_________________________________________________________________
re_lu_7 (ReLU)               (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               20100     
_________________________________________________________________
re_lu_8 (ReLU)               (None, 100)               0   

In [46]:
pred_result = model.predict(test_bow2000.toarray(), batch_size=128)
pd.DataFrame(pred_result).to_csv("08_MLP_BOW2000_epoch04_ORI.csv")
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_08_MLP_BOW2000_epoch04_ORI.csv")

### BOW2000 + ORI DATA

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

label_encoder = LabelEncoder()
label_encoder.fit(y_train_BOW500_ORI)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train_BOW2000_ORI[0:4])
print('\ny_train.shape: ', y_train_BOW2000_ORI.shape)
print('y_test.shape: ', y_valid_BOW2000_ORI.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train = label_encode(label_encoder, y_train_BOW2000_ORI)
Y_test = label_encode(label_encoder, y_valid_BOW2000_ORI)

print('\n\n## After convert')
print('y_train[0:4]:\n', Y_train[0:4])
print('\ny_train.shape: ', Y_train.shape)
print('y_test.shape: ', Y_test.shape)

# I/O check
input_shape = X_train_BOW2000_ORI.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

In [47]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger
import tensorflow.python.keras.backend as K 
sess = K.get_session()

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=400)(X)
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=200)(H1)
H2 = ReLU()(H1_W2)

# 3nd hidden layer
H2_W3 = Dense(units=100)(H2)
H3 = ReLU()(H2_W3)

# output layer
H3_W4 = Dense(units=output_shape)(H3)
H4 = Softmax()(H3_W4)

model_output = H4

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

csv_logger = CSVLogger('logs/training_log_09.csv')

# training setting
epochs = 4
batch_size = 32
# X_train_reorder = X_train.sort_indices
# training!
history = model.fit(X_train_BOW2000_ORI, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_valid_BOW2000_ORI, Y_test))

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 2000)]            0         
_________________________________________________________________
dense_12 (Dense)             (None, 400)               800400    
_________________________________________________________________
re_lu_9 (ReLU)               (None, 400)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 200)               80200     
_________________________________________________________________
re_lu_10 (ReLU)              (None, 200)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               20100     
_________________________________________________________________
re_lu_11 (ReLU)              (None, 100)               0   

In [48]:
pred_result = model.predict(test_bow2000_ORI.toarray(), batch_size=128)
pd.DataFrame(pred_result).to_csv("09_MLP_BOW2000_ORI_epoch04_ORI.csv")
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_09_MLP_BOW2000_ORI_epoch04_ORI.csv")

## 3.6 BI-LSTM

In [49]:
import tensorflow as tf
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.callbacks import CSVLogger

BILSTM_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, 256, input_length=100),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(CuDNNLSTM(100, return_sequences=True)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(CuDNNLSTM(100)),

    tf.keras.layers.Dense(8, activation='softmax')
])

csv_logger = CSVLogger('logs/training_log_10.csv')

BILSTM_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

BILSTM_model.summary()

history_BILSTM = BILSTM_model.fit(
                    X_train_keras, y_train_keras,
                    epochs=4,
                    batch_size=32, 
                    callbacks=[csv_logger],
                    validation_data = (X_valid_keras, y_valid_keras)
)

BILSTM_model.save('10_BILSTM_epoch4_length_100.h5', save_format='tf')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 256)          2560000   
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 200)          286400    
______________________________________________

2022-11-24 04:32:33.037479: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


1164450/1164450 [==============================] - 1309s 1ms/sample - loss: 1.3026 - acc: 0.5299 - val_loss: 1.2530 - val_acc: 0.5464
Epoch 2/4
1164450/1164450 [==============================] - 1304s 1ms/sample - loss: 1.2300 - acc: 0.5556 - val_loss: 1.2317 - val_acc: 0.5538
Epoch 3/4
1164450/1164450 [==============================] - 1304s 1ms/sample - loss: 1.2011 - acc: 0.5665 - val_loss: 1.2279 - val_acc: 0.5564
Epoch 4/4
1164450/1164450 [==============================] - 1307s 1ms/sample - loss: 1.1844 - acc: 0.5722 - val_loss: 1.2284 - val_acc: 0.5565


In [52]:
BILSTM_result = BILSTM_model.predict(X_test_keras, batch_size=32)
pd.DataFrame(BILSTM_result).to_csv("10_BILSTM_epoch4_length_100.csv") # including all emotions
BILSTM_pred_list = np.argmax(BILSTM_result, axis=1)
id_to_emotion = lambda labels: np.array([index_to_emotions.get(x) for x in BILSTM_pred_list])
BILSTM_id_to_emotion = id_to_emotion(BILSTM_pred_list)
output = test_dataset[['tweet_id']].reset_index(drop=True)
output['emotion']=pd.Series(BILSTM_id_to_emotion)
output.columns = ['id', 'emotion']
output.to_csv('10_result.csv', index=False) # emotion result only

In [53]:
import tensorflow as tf
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.callbacks import CSVLogger

BILSTM_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, 256, input_length=100),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(CuDNNLSTM(100, return_sequences=True)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(CuDNNLSTM(100)),

    tf.keras.layers.Dense(8, activation='softmax')
])

csv_logger = CSVLogger('logs/training_log_11.csv')

BILSTM_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

BILSTM_model.summary()

history_BILSTM = BILSTM_model.fit(
                    X_train_keras_ORI, y_train_keras_ORI,
                    epochs=4,
                    batch_size=32, 
                    callbacks=[csv_logger],
                    validation_data = (X_valid_keras_ORI, y_valid_keras_ORI)
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 256)          2560000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 200)          286400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 200)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               241600    
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 1608      
Total params: 3,089,608
Trainable params: 3,089,608
Non-trainable params: 0
____________________________________________

In [54]:
BILSTM_result = BILSTM_model.predict(X_test_keras, batch_size=32)
pd.DataFrame(BILSTM_result).to_csv("11_BILSTM_ORI_epoch4_length_100.csv")
BILSTM_pred_list = np.argmax(BILSTM_result, axis=1)
id_to_emotion = lambda labels: np.array([index_to_emotions.get(x) for x in BILSTM_pred_list])
BILSTM_id_to_emotion = id_to_emotion(BILSTM_pred_list)
output = test_dataset[['tweet_id']].reset_index(drop=True)
output['emotion']=pd.Series(BILSTM_id_to_emotion)
output.columns = ['id', 'emotion']
output.to_csv('11_result.csv', index=False)

----
# Report

I've aleady re-build this jupyter notebook for typesetting, so some value may be different.

## 1 Compare the result

(all score are based on kaggle)

- Different Classification model with tokenizer

    - Decision Tree + BOW500(tokenizer=nltk.word_tokenize)

        The score of this is 0.18764 only, so I give it up in early stage.

    - Random Forest + BOW500(tokenizer=nltk.word_tokenize)

        The score of this is 0.30764.

        Firstly, I think it's a good direction to dig into. However, when I check the output, I found that it outputs "joy" to all tweats. It's obviously abnormal, so I give up this way.

    - Multinomial Naive Bayes + TFIDF(default tokenizer)

        The score with clean data is 0.30083, and the score with original data is 0.31127.

        In this case, I found that the original data's performance is better than the data I cleaned. I worried about that I clean some meaningful features, but the case I did I thaught is normal. Therefore, in all the next steps, I will also implement 2 kinds of data to predict the result.

    - XGB Classifier + TFIDF(default tokenizer)

        The score with clean data is 0.29176, because it performed wroser than MNB, and it requires 2 hours to train, so I give up test it on original data.

        I found this method on kaggle, and many people consider it as a state-of-the-art way to competiotion, but it seems that I can't control it.

    -  MLP + BOW500/2000(tokenizer=nltk.word_tokenize)

        For BOW500, the score with clean data is 0.41512, and the score with original data is 0.43921.

        For BOW2000, the score with clean data is 0.4168, and the score with original data is 0.45372.

        The model I build can refer to the summary of the cell, or describe below:

        input->Dense400+Relu->Dense200+Relu->Dense100+Relu->Dense8+softmax->output

        I found that if we create more bags of words, it seems that it can enhance the performance.

        Nonetheless, when I tried BOW3000 of original data, it comes out that the score is 0.43921, it's worsen.

        Besides, the overfitting is coming out at epoch 4. I tried the BOW2000+original data in epoch 4 and 12 respectively, it comes out the result is 0.45372 and 0.43731.

        ![Training Loss per epoch in BOW2000](img/training_log_epoch12.png)

        Therefore, in my case, MLP+BOW2000 at epoch 4 is the best result.

    -  BILSTM + keras.tokenizer(tokenizer=nltk.word_tokenize)

        I found some competiotion report that uses BI-LSTM as a powerful tool, so I tried to implement it.

        Because in the above test, I found that original data's performance is better than my clean data, I only tried on original data.

        For default length + original data, the score is *0.48345, which is my highest score.*

        For length 400, the score is 0.43641, for length 100, the score is 0.43653. Therefore, it seems that the difference is not obvious.

        Conclusively, due to lack of time, I still have trouble controling this powerful method, it's quite unfortunately.


## 2 The bad situation I faced

- Missinput the wrong index label

    It consumes me many time to figure out why my grade is so slow; eventually, when I tried to sort them to compare the result, I found that the tweat_id of my test_dataset are different in different kernal.

    While I fixed this bug, the grade becomes normal, but it consumes me lots of time to re-train the data to fix it.


## 3 The thing I can improve

- Comprehensive the parameter of BI-LSTM to improve it.

- Try on specific NLP model. (i.e., BERT, RoBertA)

